# Data Preprocessing

This notebook is the base for :
- Data cleaning
- Data Impuation
- Feature Engineering

## Used libraries

In [295]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import skew, kurtosis

## Loading data

The train and test inputs are composed of 46 features.

The target of this challenge is `RET` and corresponds to the fact that the **return is in the top 50% of highest stock returns**.

Since the median is very close to 0, this information should not change much with the idea to predict the sign of the return.

In [ ]:
x_train = pd.read_csv('../x_train.csv', index_col='ID')
y_train = pd.read_csv('../y_train.csv', index_col='ID')
train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('../x_test.csv', index_col='ID')
train.head()

## Data Preprocessing
- Cleaning : removing all rows with no observed returns over the past 5 days
- Imputation : Simple Impute the median for the remaining NaNs of RET_x and VOLUME_x


In [306]:
# Drop all rows with no observed returns over the past 5 days
ret_5D_window_cols = [f'RET_{day}' for day in range(1,6)]
ret_to_drop = train[(train[ret_5D_window_cols].isna().sum(axis=1)/(train[ret_5D_window_cols].shape[1]) >= 1)][ret_5D_window_cols]

train.drop(index=ret_to_drop.index, inplace=True) 

# Simple Impute the median for the remaining NaNs of RET_x and VOLUME_x
ret_cols = [col for col in train.columns if 'RET_' in col]
volume_cols = [col for col in train.columns if 'VOLUME_' in col]
impute_features = ret_cols + volume_cols
imputer = SimpleImputer(strategy='median')
train[impute_features] = imputer.fit_transform(train[impute_features])


## Feature Engineering

The main drawback in this challenge would be to deal with the noise. To do that, we could create some feature that aggregate features with some statistics. 

The following cell computes statistics on a given target conditionally to some features. For example, we want to generate a feature that describe the mean of `RET_1` conditionally to the `SECTOR` and the `DATE`.

**Ideas of improvement**: change shifts, the conditional features, the statistics, and the target. 

In [ ]:
# Feature engineering
new_features = []

# Aggregating features and their differences w.r.t. the features

shifts = [1,2,3,4,5] 
statistics = {'mean':'mean', 'std':'std','skew': lambda x: skew(x, nan_policy='omit'), 'kurt': lambda x: kurtosis(x, nan_policy='omit'), 'max':'max', 'min':'min'}
gb_features_list = [['SECTOR', 'DATE'], ['INDUSTRY_GROUP', 'DATE'], ['INDUSTRY', 'DATE'], ['SUB_INDUSTRY', 'DATE']]
target_features = ['RET','VOLUME']
for target_feature in target_features:
    for [cat, date] in gb_features_list:
        tmp_name = cat + '_' + date
        for shift in shifts:
            for stat_name,stat in statistics.items():
                name = f'{target_feature}_{shift}_{tmp_name}_{stat_name}'
                feat = f'{target_feature}_{shift}'
                new_features.append(name)
                for data in [train, test]:
                    data[name] = data.groupby([cat, date])[feat].transform(stat)

# Cumulative returns over the past x days
days = [2,3,4,5]

for day in days:
    name = f'cumulative_RET_{day}D'
    new_features.append(name)
    for data in [train, test]: # formula product of 1+ret for each day minus 1
        data[name] = data[[f'RET_{i}' for i in range(1,day+1) ]].apply(lambda x: (1+x).prod(), axis=1) - 1

weeks = 4


# TODO : Add the kurtosis and skewness of the returns and volumes
target_features = ['RET', 'VOLUME'] 
for target_feature in target_features:
    for week in range(weeks):
        name = f'{target_feature}_STOCK_WEEK_{week+1}'
        new_features.append('mean_' + name)
        new_features.append('std_' + name)
        for data in [train, test]:
            data['mean_' + name] = data[[f'{target_feature}_{week*5 + day}' for day in range(1,6)]].mean(axis=1)
            data['std_' + name] = data[[f'{target_feature}_{week*5 + day}' for day in range(1,6)]].std(axis=1)
       
# TODO : Change the way the features is created because the DATE agregation is not relevant for dealing with the differents weeks
shifts = [1,2,3,4] 
statistics = ['sum']
gb_features_list = [['SECTOR', 'DATE']]
target_features = ['mean_VOLUME_STOCK_WEEK']
for target_feature in target_features:
    for gb_features in gb_features_list:
        tmp_name = '_'.join(gb_features)
        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_/total_VOLUME_of_SECTOR_DATE'
                feat = f'{target_feature}_{shift}'
                new_features.append(name)
                for data in [train, test]:
                    data[name] = data[feat]/data.groupby(gb_features)[feat].transform('sum')

shifts = [1,2,3,4] 
statistics = ['sum'] 
gb_features_list = [['SECTOR', 'DATE']]
target_features = ['mean_RET_STOCK_WEEK']
for target_feature in target_features:
    for gb_features in gb_features_list:
        tmp_name = '_'.join(gb_features)
        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_/total_RET_of_SECTOR_DATE'
                feat = f'{target_feature}_{shift}'
                new_features.append(name)
                for data in [train, test]:
                    data[name] = data[feat]/data.groupby(gb_features)[feat].transform('sum')


# TODO : Get back on this notion of momentum

weeks = [1,2,3,4]
targets = ['RET', 'VOLUME']

for target in targets:
    for week in weeks: 
        window_size = 5*week
        name = f'{target}_{window_size}_day_momentum'
        new_features.append(name)
        for data in [train, test]:
            _data = data.copy()
            rolling_mean_target = _data.groupby(by=['SECTOR', 'DATE'])[[f'{target}_{day}' for day in range(2, window_size+1)]].mean()
            target_1_mean = _data.groupby(by=['SECTOR', 'DATE'])[[f'{target}_1']].mean()
            target_1_mean_aligned, rolling_mean_target_aligned = target_1_mean.align(rolling_mean_target, axis=0, level='SECTOR')
            target_momentum = target_1_mean_aligned.sub(rolling_mean_target_aligned.mean(axis=1), axis=0)
            target_momentum.rename(columns={f'{target}_1': name},inplace=True)
            placeholder = _data.join(target_momentum, on=['SECTOR', 'DATE'], how='left')
            data[name] = placeholder[name]



weeks = [1,2,3,4]
targets = ['RET', 'VOLUME']

for week in weeks: 
    window_size = 5*week
    for target in targets: 
        name = f'{window_size}_day_mean_{target}_vola'
        new_features.append(name)
        for data in [train, test]:
            rolling_std_target = data.groupby(by=['SECTOR', 'DATE'])[[f'{target}_{day}' for day in range(1,window_size+1)]].mean().std(axis=1).to_frame(name)
            placeholder = data.join(rolling_std_target, on=['SECTOR', 'DATE'], how='left')
            data[name] = placeholder[name]

# do the same with the stock granularity or other categories of aggregation

targets = ["RET"]
window_size = [5, 10, 15, 20]

for window in window_size:
    name = f"RSI_SECTOR_DATE_{window}"
    new_features.append(name)
    for target in targets:
        for data in [train, test]:
            _data = data.copy()
            avg_gain_sector_day = _data.groupby(by=['SECTOR', 'DATE'])[[f'{"RET"}_{day}' for day in range(1, window+1)]].mean().agg(lambda x: x[x>0].mean(), axis=1)
            avg_loss_sector_day = _data.groupby(by=['SECTOR', 'DATE'])[[f'{"RET"}_{day}' for day in range(1, window+1)]].mean().agg(lambda x: x[x<0].mean(), axis=1).abs()
            rs_sector_day = avg_gain_sector_day/avg_loss_sector_day
            rsi_sector_date = 100 - 100/(1+rs_sector_day)
            placeholder = _data.join(rsi_sector_date.to_frame(name), on=['SECTOR', 'DATE'], how='left')
            data[name] = placeholder[name]

# do the same with the stock granularity or other categories of aggregation
window_size = [5, 10, 15, 20]
for window in window_size:
    name = f'ADL_{window}'
    new_features.append(name)
    for data in [train, test]:
        _data = data.copy()
        sum_adl = ((_data.groupby(by=["SECTOR", "DATE"])[[f'RET_{day}' for day in range(1, window+1)]]).apply(lambda x: (x>0).sum()) - (_data.groupby(by=["SECTOR", "DATE"])[[f'RET_{day}' for day in range(1, window+1)]]).apply(lambda x: (x<0).sum())).sum(axis=1)
        placeholder = _data.join(sum_adl.to_frame(name), on=['SECTOR', 'DATE'], how='left')
        data[name] = placeholder[name]

# ADDITIONAL FEATURES

def compute_moving_avg(df, cols):
    return df[cols].mean(axis=1)
   
def compute_volatility(df, cols):
    return df[cols].std(axis=1)

def compute_ema(df, cols, span=5):
    return df[cols].ewm(span=span, axis=1).mean().iloc[:, -1]

def compute_momentum(df, col_start, col_end):
    return df[col_end] - df[col_start]

def compute_relative_volume(df, cols, last_col):
    return df[last_col] / df[cols].median(axis=1)

def compute_rsi(df, cols):
    gains = df[cols].clip(lower=0).mean(axis=1)
    losses = df[cols].clip(upper=0).abs().mean(axis=1)
    return 100 - (100 / (100 + (gains / losses)))

def compute_vw_ret(df, ret_cols, vol_cols):
    return (df[ret_cols] * df[vol_cols]).sum(axis=1) / df[vol_cols].sum(axis=1)

def generate_indicators(df, apply_to=['RET', 'VOLUME'], num_days = 20):
    ind_columns = {}

    feature_functions = {
        'MA': compute_moving_avg,
        'VOLATILITY': compute_volatility,
        'EMA': compute_ema,
        'MOMENTUM': compute_momentum,
        'REL_VOL': compute_relative_volume,
        'RSI': compute_rsi
    }
    
    for feature in apply_to:
        cols = [f'{feature}_{i}' for i in range(1, num_days+1)]
        ind_columns[f'MA_{feature}'] = feature_functions['MA'](df, cols)
        ind_columns[f'VOLATILITY_{feature}'] = feature_functions['VOLATILITY'](df, cols)
        ind_columns[f'EMA_{feature}'] = feature_functions['EMA'](df, cols)
        ind_columns[f'MOMENTUM_{feature}'] = feature_functions['MOMENTUM'](df, f'{feature}_1', f'{feature}_{num_days}')
        
        if feature == 'VOLUME':
            ind_columns['REL_VOL'] = feature_functions['REL_VOL'](df, cols, f'{feature}_{num_days}')
        
        if feature == 'RET':
            ind_columns['RSI_RET'] = feature_functions['RSI'](df, cols)

    if 'RET' in apply_to and 'VOLUME' in apply_to:
        ind_columns['VW_RET'] = compute_vw_ret(df, [f'RET_{i}' for i in range(1, num_days+1)], [f'VOLUME_{i}' for i in range(1, num_days+1)])

    for shift in [5, 10, 20]:
        ret, vol = df[[f'RET_{i}' for i in range(1, shift+1)]], df[[f'VOLUME_{i}' for i in range(1, shift+1)]]
        # returns statistics
        ind_columns[f'Mean_RET_{shift}'] = np.mean(ret, axis=1)
        ind_columns[f'Std_RET_{shift}'] = np.std(ret, axis=1)
        ind_columns[f'Range_RET_{shift}'] = (lambda x: np.max(x, axis=1) - np.min(x, axis=1))(ret)
        ind_columns[f'Momentum_RET_{shift}'] = ret.iloc[:, -1] - ret.iloc[:, 0]
        ind_columns[f'Cumulative_RET_{shift}'] = np.prod(1 + ret, axis=1) - 1
        ind_columns[f'Skew_RET_{shift}'] = skew(ret, nan_policy='omit', axis=1)
        ind_columns[f'Kurtosis_RET_{shift}'] = kurtosis(ret, nan_policy='omit', axis=1)

        # volumes statistics
        ind_columns[f'Mean_VOL_{shift}'] = np.mean(vol, axis=1)
        ind_columns[f'Std_VOL_{shift}'] = np.std(vol, axis=1)
        ind_columns[f'Skew_VOL_{shift}'] = skew(vol, nan_policy='omit', axis=1)
        ind_columns[f'Kurtosis_VOL_{shift}'] = kurtosis(vol, nan_policy='omit', axis=1)
        # correlation between returns and volumes
        ind_columns[f'Corr_RET_VOL_{shift}'] = [np.corrcoef(ret.iloc[i], vol.iloc[i])[0, 1] for i in range(len(ret))]

    # compute volume surge (VOL_1 - Mean_VOL_20) / Std_VOL_20
    ind_columns['VOL_SURGE_5'] = (df['VOLUME_1'] - ind_columns['Mean_VOL_5']) / ind_columns['Std_VOL_5']
    ind_columns['VOL_SURGE_10'] = (df['VOLUME_1'] - ind_columns['Mean_VOL_10']) / ind_columns['Std_VOL_10']
    ind_columns['VOL_SURGE_20'] = (df['VOLUME_1'] - ind_columns['Mean_VOL_20']) / ind_columns['Std_VOL_20']
     
    return ind_columns

## Outputing the extended dataframe

In [ ]:
train = pd.concat([train, pd.DataFrame(generate_indicators(train))], axis=1)
test = pd.concat([test, pd.DataFrame(generate_indicators(test))], axis=1)

# Save the new datasets
train.to_csv('../train_extended.csv')
test.to_csv('../test_extended.csv')